In [ ]:
# Opening catalogue and reading data
with open("radius4_catalogue.txt", "r") as file:
    data = file.read()  
print(data)


In [ ]:
#Extract magnitude values 
data_mag = []  # Initialize a list to store magnitude values
with open("radius4_catalogue.txt", "r") as file:
    next(file)  # skipping header
    for line in file:
        columns = line.split()
        if len(columns) >= 7:  
            data_mag.append(float(columns[6]))  # Extract relevant column and turn into floats


In [ ]:
#Extract magnitude error values 
error_mag = []
with open("radius4_catalogue.txt", "r") as file:
    next(file)  # skipping header
    for line in file:
        columns = line.split()
        if len(columns) >= 8:
            error_mag.append(float(columns[7]))  # Extract relevant column and turn into floats
print(np.mean(error_mag))#Check mean of error values 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create empty lists to store N(m) and m
thresholds = []
counts = []

# Loop through data for certain magnitude thresholds 
for threshold in np.arange(0, 25, 0.25): #Range of magnitudes taken
    count = len([value for value in data_mag if value <= threshold])
    thresholds.append(threshold)
    counts.append(count)

# Need to convert to log(N(m))
log_counts = []
errors_log_counts = []

for count in counts:
    if count > 0:
        log_count = np.log10(count)
        poisson_error = np.sqrt(count)  #poisson error
        log_error = poisson_error / (count * np.log(10))  # error propogation
    else:
        log_count = np.nan
        log_error = np.nan
    
    log_counts.append(log_count)
    errors_log_counts.append(log_error)

# Check validity of data to avoid error message
valid_indices = ~np.isnan(log_counts)
valid_thresholds = np.array(thresholds)[valid_indices]
valid_log_counts = np.array(log_counts)[valid_indices]
valid_errors_log_counts = np.array(errors_log_counts)[valid_indices]

# Filter data up to m = 19.50 for the linear fit before data flattens
fit_threshold = 19.50
fit_indices = valid_thresholds <= fit_threshold
fit_thresholds = valid_thresholds[fit_indices]
fit_log_counts = valid_log_counts[fit_indices]
fit_errors_log_counts = valid_errors_log_counts[fit_indices]

# Linear fit on data 
fit_coefficients = np.polyfit(fit_thresholds, fit_log_counts, 1)
fit_slope, fit_intercept = fit_coefficients
fit_line = np.polyval(fit_coefficients, valid_thresholds)  

# 'Sliding window slope analysis' verifies where the graph flattena
window_size = 5 
horizontal_slope_threshold = 0.01 
first_horizontal_m = None

for i in range(len(valid_thresholds) - window_size + 1):
    window_thresholds = valid_thresholds[i:i + window_size]
    window_log_counts = valid_log_counts[i:i + window_size]
    coefficients = np.polyfit(window_thresholds, window_log_counts, 1)#fits line to get slope
    slope = coefficients[0]
    
    # Check if the slope is approximately zero
    if abs(slope) < horizontal_slope_threshold:
        first_horizontal_m = window_thresholds[0]
        break

# Plot the data with error bars and linear fit
plt.figure(figsize=(8, 6))
plt.errorbar(valid_thresholds, valid_log_counts, yerr=valid_errors_log_counts, fmt='o', color='blue', label="Log(N(m) > m)", capsize=3)
plt.plot(valid_thresholds, fit_line, linestyle='--', color='orange', label=f"Linear Fit (m ≤ {fit_threshold})\nlog(N(m)) = {fit_slope:.2f}m - {-1*fit_intercept:.2f}")
plt.xlabel("Magnitude", fontsize=14)
plt.ylabel("Log(Number of Sources above Magnitude)", fontsize=14)
plt.title("Log(Number of Sources above Magnitude) vs. Magnitude", fontsize=16)
plt.grid(True, linestyle="--", alpha=0.7)
#Plot line to display where graphflattens
if first_horizontal_m is not None:
    plt.axvline(first_horizontal_m, color='red', linestyle='--', label=f"Magnitude when grad. is approx. 0 ≈ {first_horizontal_m:.2f}")
plt.legend()
plt.savefig('LOGN V M')#Save plot
plt.show()
#Prints value of m when line flattens
if first_horizontal_m is not None:
    print(f"First magnitude limit for horizontal line: {first_horizontal_m:.2f}")
else:
    print("No horizontal line detected.")

    
#Calculating error of slope and intercept 
coefficients = np.polyfit(valid_thresholds, valid_log_counts, 1) 
slope, intercept = coefficients
residuals = valid_log_counts - np.polyval(coefficients, valid_thresholds)#calculate residuals
residual_variance = np.sum(residuals**2) / (len(valid_log_counts) - 2)#calculate variance
delta = len(valid_log_counts) * np.sum(valid_thresholds**2) - (np.sum(valid_thresholds))**2
slope_uncertainty = np.sqrt(residual_variance * len(valid_log_counts) / delta)
intercept_uncertainty = np.sqrt(residual_variance * np.sum(valid_thresholds**2) / delta)

# Print the slope, intercept, and their uncertainties
print(f"{slope_uncertainty:.2f}")
print(f"{intercept_uncertainty:.2f}")
